#Use isto se for a primeira vez usando o datalake da basedosdados.org: <br>
   pip install basedosdados
<br><br>
#Use para baixar a base como microdados_violencia.csv: <br>
   import basedosdados as bd <br>
   df = bd.read_table(dataset_id='br_ms_sinan', table_id='microdados_violencia', billing_project_id="violenciasinan", use_bqstorage_api = True) <br>
   df.to_csv(path_or_buf = 'microdados_violencia.csv')

In [24]:
# importar bibliotecas usadas
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.simplefilter('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental import preprocessing

In [25]:
print(tf.__version__)
try:
    physical_devices = tf.config.list_physical_devices('GPU') 
    #tf.config.experimental.set_memory_growth(physical_devices[0], True)
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print(physical_devices)
except:
    print("No GPU")

2.9.1
[]


In [27]:
# carregar dados em um dataframe pandas
# para usar o arquivo .csv baixe da pasta do sharepoint ou com os comandos disponíveis no topo do notebook
df = pd.read_csv("C:/Users/rozan/OneDrive - Fundacao Getulio Vargas - FGV/Eletivas/AdvLearning/microdados_violencia.csv")
df.drop(['houve_qual_outra_violencia_sexual', 'meio_qual_outro', 'ocorreu_qual_outra', 'outro_local_ocorrencia', 'quais_outras_deficiencias_paciente', 'autor_relacao_outros'], axis=1)

,Unnamed: 0,ano,data_notificacao,id_categoria_cid10,id_subcategoria_cid10,tipo_notificacao,id_uf_notificacao,id_municipio_notificacao,id_municipio_6_notificacao,id_unidade_notificacao,...,encaminhamento_conselho_idoso,encaminhamento_delegacia_idoso,encaminhamento_direitos_humanos,encaminhamento_mpu,encaminhamento_delegacia_crianca,encaminhamento_delegacia_mulher,encaminhamento_delegacia,encaminhamento_justica_infancia_juventude,encaminhamento_defensoria_publica,data_encerramento
0,0,2009,2009-11-03,Y09,Y00.4,2,12,1200104.0,120010,2001500.0,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,2009-11-04
1,1,2009,2009-02-04,Y09,NaN,2,27,2700300.0,270030,3015408.0,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,2009-02-04
2,2,2009,2009-03-18,Y09,NaN,2,27,2700300.0,270030,3015408.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-03-20
3,3,2009,2009-03-31,Y09,NaN,2,27,2700300.0,270030,3015408.0,...,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,2009-03-31
4,4,2009,2009-07-23,Y09,NaN,2,27,2700300.0,270030,3015408.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-07-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299137,2299137,2016,2016-02-03,Y09,NaN,2,33,3302403.0,330240,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2016-02-03
2299138,2299138,2016,2016-02-04,Y09,NaN,2,33,3302403.0,330240,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2016-03-01
2299139,2299139,2016,2016-02-10,Y09,NaN,2,33,3302502.0,330250,2278332.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016-03-21
2299140,2299140,2016,2016-02-17,Y09,Y06.1,2,33,3304557.0,330455,2296616.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2016-02-17


In [ ]:
# resumo dos dados
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df)

In [ ]:
# separando labels e features, features transformadas em array
features = df.copy()
labels = features.pop('outras_vezes_ocorrencia')
# Create a symbolic input
input = tf.keras.Input(shape=(), dtype=tf.float32)

# Do a calculation using is
result = 2*input + 1

# the result doesn't have a value
result
calc = tf.keras.Model(inputs=input, outputs=result)

In [ ]:
print(calc(1).numpy())
print(calc(2).numpy())

In [ ]:
inputs = {}

for name, column in features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

inputs

In [ ]:
# concatenar e normalizar dados numéricos
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = preprocessing.Normalization()
norm.adapt(np.array(df[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)
preprocessed_inputs = [all_numeric_inputs]

all_numeric_inputs

In [ ]:
# índices para dados em string
for name, input in inputs.items():
  if input.dtype == tf.float32:
    continue
  
  lookup = preprocessing.StringLookup(vocabulary=np.unique(features[name]))
  one_hot = preprocessing.CategoryEncoding(max_tokens=lookup.vocab_size())

  x = lookup(input)
  x = one_hot(x)
  preprocessed_inputs.append(x)

In [ ]:
# modelo de regressão
# SGD como otimizador
modelo = tf.keras.Sequential([
  layers.Dense(64),
  layers.Dense(1)
])

modelo.compile(loss = tf.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.SGD())

In [ ]:
modelo.fit(features, labels, epochs=10)